In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, uint
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

@si.coroutine(inputs={"I" : si.Bit})
def Detect111():
    cnt = uint(0, 2)
    I = yield
    while True:
        if (I):
            if (cnt<3):
                cnt = cnt+1
        else:
            cnt = 0
        O = (cnt == 3)
        I = yield O
        
detect111 = si.compile(Detect111(), file_name="build/silica_detect111.py")
with open("build/silica_detect111.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width, strategy):
    if strategy == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
               

In [2]:
!magma -o coreir -m coreir -t Detect111 build/silica_detect111.py
!coreir -i build/silica_detect111.json -o build/silica_detect111.v
!yosys -p 'synth_ice40 -top Detect111 -blif build/silica_detect111.blif' build/silica_detect111.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_detect111.txt build/silica_detect111.blif
!icetime -tmd hx1k build/silica_detect111.txt  | grep -B 2 "Total path delay"

In Run Generators

Modified?: Yes
2.27. Printing statistics.

=== Detect111 ===

   Number of wires:                198
   Number of wire bits:            264
   Number of public wires:         196
   Number of public wire bits:     262
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  8
     SB_DFF                          4
     SB_LUT4                         4

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_detect111.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          6 / 1280
  DFF        4
  CARRY      0
  CARRY, DFF 0
  DFF PASS   2
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 0 nets
 

In [3]:
!yosys -p 'synth_ice40 -top detect111 -blif build/verilog_detect111.blif' ../verilog/detect111.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_detect111.txt build/verilog_detect111.blif
!icetime -tmd hx1k build/verilog_detect111.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== detect111 ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           4
   Number of public wire bits:       5
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  7
     SB_DFFESR                       2
     SB_LUT4                         5

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_detect111.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          5 / 1280
  DFF        2
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 0 nets
  0 globals
realize_constants...
p